In [1]:
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D,Flatten,MaxPooling2D,ZeroPadding2D,Concatenate,Lambda,Softmax,GlobalAveragePooling1D,MaxPooling1D,SpatialDropout1D,ReLU, Dense, Activation,Reshape,BatchNormalization, add, Embedding,Conv1D,LayerNormalization,MultiHeadAttention,Add,Dropout,Layer
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import load_model
from Uilts import *
from capsulelayers import *

In [2]:

with h5py.File('./Dataset/data_train.h5', 'r') as hf:
    datall = hf['datall'][:]     
    laballx = hf['laballx'][:]  

print("datall shape:", datall.shape)   
print("laballx shape:", laballx.shape) 

laball = tf.keras.utils.to_categorical(laballx, num_classes=2)

print(len(np.where(laballx == 0)[0]), len(np.where(laballx == 1)[0]))  


datall shape: (6444, 20, 6000, 3)
laballx shape: (6444,)
3169 3275


In [3]:
# Split dataset
from sklearn.model_selection import train_test_split

# np.random.seed(42)
# io = np.random.permutation(len(datall))
# datall = datall[io]
# laball = laball[io]

sp = 0.1
x_train, x_test, y_train, y_test = train_test_split(datall, laball, test_size=sp, random_state=2024)
print(x_test.shape)
print(y_train.shape)

(645, 20, 6000, 3)
(5799, 2)


In [4]:
model = load_model('Epochs/best_model.h5',custom_objects={'margin_loss': margin_loss,'CapsuleLayer':CapsuleLayer,'Length':Length,'PatchEncoder':PatchEncoder,'Patches':Patches,'StochasticDepth':StochasticDepth})
outte = model.predict([x_test],batch_size=16,verbose=1)

Instructions for updating:
Use fn_output_signature instead
41/41 [==============================] - 3s 29ms/step


In [5]:
from sklearn.metrics import confusion_matrix
print(x_test.shape)
labtestx = y_test
outtest = np.argmax(outte,axis=-1)
labtestx= np.argmax(labtestx,axis=-1)

cf_matrix = confusion_matrix(labtestx,outtest)
print('cf_matrix:',cf_matrix)

(645, 20, 6000, 3)
cf_matrix: [[303   3]
 [  5 334]]


In [6]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
outtest = np.array(outtest)
labtestx = np.array(labtestx)
print('accuracy:',accuracy_score(labtestx, outtest))
print('precision_score:',precision_score(labtestx, outtest))
print('recall_score:',recall_score(labtestx, outtest))
print('f1_score:',f1_score(labtestx, outtest))

accuracy: 0.9875968992248062
precision_score: 0.9910979228486647
recall_score: 0.9852507374631269
f1_score: 0.9881656804733727
